In [1]:
import pandas as pd

df = pd.read_csv(
    "noticias_unificadas.tsv",
    encoding="utf-8",
    sep="\t",
    dtype={"fecha": "string", "titulo": "string", "contenido": "string", "seccion": "string", "link": "string"},
    quoting=0,
    na_filter=False
)

In [2]:
topicos = df['seccion'].value_counts()
print(f"\nTotal de tópicos únicos: {df['seccion'].nunique()}")


Total de tópicos únicos: 7


In [3]:
df.head()

,fecha,titulo,contenido,seccion,link
0,2025-11-09,jueces rechazan intento de afectacion a la ind...,desde la ciudad de tacna jueces y juezas de to...,Política,https://diariocorreo.pe/politica/jueces-rechaz...
1,2025-11-09,liga lo gritan los churres y todo el pueblo de...,alianza atletico le saco lustre a su clasifica...,Deportes,https://diariocorreo.pe/deportes/alianza-atlet...
2,2025-11-09,proponen sancionar con hasta anos de carcel a ...,la congresista elizabeth medina hermosillo de ...,Política,https://diariocorreo.pe/politica/proponen-sanc...
3,2025-11-09,este lunes inicia la semana de representacion ...,desde este lunes hasta el viernes de noviembre...,Política,https://diariocorreo.pe/politica/este-lunes-in...
4,2025-11-09,seleccion peruana evalua reprogramacion de par...,la federacion peruana de futbol fpf informo qu...,Deportes,https://diariocorreo.pe/deportes/seleccion-per...


In [4]:
import nltk
from nltk import trigrams
from nltk.tokenize import sent_tokenize, word_tokenize


In [5]:
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/joelibaceta/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/joelibaceta/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [6]:
nltk.data.find("tokenizers/punkt")

FileSystemPathPointer('/home/joelibaceta/nltk_data/tokenizers/punkt')

In [7]:
nltk.data.find("tokenizers/punkt_tab")

FileSystemPathPointer('/home/joelibaceta/nltk_data/tokenizers/punkt_tab')

In [8]:
from typing import Dict, List, Tuple

def tokenize_es(text: str) -> list[list[str]]:
    if not isinstance(text, str) or not text.strip():
        return []
    sentences = sent_tokenize(text, language="spanish")
    tokenized = []
    for s in sentences:
        toks = word_tokenize(s, language="spanish")
        # lower only alphabetic tokens, keep punctuation as-is
        toks = [t.lower() if t.isalpha() else t for t in toks]
        if toks:
            tokenized.append(toks)
    return tokenized

In [9]:
from collections import defaultdict

def load_corpus(df: pd.DataFrame, by_category: bool = False) -> Dict[str, List[List[str]]]:
    buckets: Dict[str, List[List[str]]] = defaultdict(list)
    
    for _, row in df.iterrows():
        categoria = (row.get("seccion") or "").strip()
        noticia = row.get("contenido")
        
        if not noticia or not isinstance(noticia, str):
            continue
            
        sents_toks = tokenize_es(noticia)
        
        if by_category and categoria:
            for s in sents_toks:
                if s:
                    buckets[categoria].append(s)
        else:
            for s in sents_toks:
                if s:
                    buckets["_GLOBAL"].append(s)
    
    return buckets

In [10]:
corpus_por_categoria = load_corpus(df, by_category=True)
corpus_global = load_corpus(df, by_category=False)

In [11]:
def train_trigrams(tokenized_sentences: List[List[str]]):
    """
      model[(w1,w2)][w3] = prob
    """
    model = defaultdict(lambda: defaultdict(float))
    for sent in tokenized_sentences:
        for w1, w2, w3 in trigrams(sent, pad_left=True, pad_right=True):
            model[(w1, w2)][w3] += 1.0

    for w1w2 in model:
        total = sum(model[w1w2].values())
        if total > 0:
            for w3 in model[w1w2]:
                model[w1w2][w3] /= total
    return model

In [12]:
model_global = train_trigrams(corpus_global["_GLOBAL"])
len(model_global)

2343647

In [13]:
models_por_categoria = {}

for categoria, oraciones in corpus_por_categoria.items():
    print(f"Entrenando modelo para: {categoria} ({len(oraciones)} oraciones)")
    models_por_categoria[categoria] = train_trigrams(oraciones)


Entrenando modelo para: Política (12509 oraciones)
Entrenando modelo para: Deportes (4739 oraciones)
Entrenando modelo para: Espectáculos (6386 oraciones)
Entrenando modelo para: Cultura (3256 oraciones)
Entrenando modelo para: Economía (3168 oraciones)
Entrenando modelo para: Mundo (5186 oraciones)
Entrenando modelo para: Policiales (2502 oraciones)


In [14]:
from typing import Optional
import random

def sample_next(model: dict, w1: str, w2: str) -> Optional[str]:
    dist = model.get((w1, w2), {})
    # Heuristica por si no hay distribución para (w1, w2)
    if not dist:
        # Fallback 1: Buscar todos los contextos que terminen en w2
        for key in model.keys():
            if key[1] == w2 and model[key]:
                dist = model[key]
                break
        # Fallback 2: Si aún no hay nada, usar inicio de oración
        if not dist:
            dist = model.get((None, None), {})
        # Fallback 3: Si todavía no hay nada, elegir un contexto aleatorio
        if not dist and model:
            random_key = random.choice(list(model.keys()))
            dist = model[random_key]

    r = random.random()
    acc = 0.0
    for w3, p in dist.items():
        acc += p
        if acc >= r:
            return w3
    
    return list(dist.keys())[-1] if dist else None

In [15]:
def generate_sentence(model: dict, seeds: Tuple[str, str] = (None, None), max_len: int = 30) -> str:
    text: List[str] = [seeds[0], seeds[1]]
    sentence_finished = False

    while not sentence_finished and len(text) < max_len + 2:
        w3 = sample_next(model, text[-2], text[-1])
        text.append(w3)
        if text[-2:] == [None, None] or w3 is None:
            sentence_finished = True

    sentence = " ".join([t for t in text if t])
    sentence = sentence.strip()
    if sentence and sentence[-1] not in ".!?":
        sentence += "."
    if sentence:
        sentence = sentence[0].upper() + sentence[1:]
    return sentence

In [16]:
def generate_paragraph(model: dict, n_sentences: int = 3, seeds: Tuple[str, str] = (None, None)) -> str:
    return " ".join(generate_sentence(model, seeds=seeds) for _ in range(n_sentences))

In [17]:
print(generate_paragraph(model_global, n_sentences=2, seeds=("La", "Policia", )))

La Policia la reserva en canal n la repite asi sea le interesa debo tener algo de nerviosismo y saltaron cuando entre y por whatsapp nuestro periodico digital enriquecido peru epaper ahora. La Policia la victoria se convertiria en su idiosincrasia ama la comida y merchandising como ropa calzado gastronomia paneton y chocolate peruano cacao litro de sangre su pais para todos los lugares.


In [18]:
print(generate_paragraph(models_por_categoria["Deportes"], n_sentences=2, seeds=("El", "equipo")))

El equipo de todos en dicha prueba el objetivo alianza lima puntos universitario se mantiene en el bien de todos llegara este sabado en un proceso clasificatorio cuidado que haces asi haya. El equipo de arequipa el en lo mio y el volante creativo llevara la camiseta ha sido el ganador estuvo dirigido por antonio rizola buscara sumar los tres volcanes mas emblematicos de.


In [19]:
print("\nPOLÍTICA:")
print(generate_paragraph(models_por_categoria["Política"], n_sentences=2, seeds=("El", "presidente")))


POLÍTICA:
El presidente del congreso control del mp victor cubas por su gobierno entregara el rolex fue al bano lo encontro en ella el premier advirtio que el comite tecnico mientras que el. El presidente del consejo de ministros pcm modifico el monto final de la ciudadania para esclarecer los hechos denunciados y expreso su preocupacion por la procuraduria.


In [20]:
from utils.utils import format_sentence

def generate_sentence(model: dict, seeds: Tuple[str, str] = (None, None), max_len: int = 30) -> str:
    w1 = seeds[0].lower() if seeds[0] and seeds[0] is not None else None
    w2 = seeds[1].lower() if seeds[1] and seeds[1] is not None else None
    
    text: List[str] = []

    if w1 is not None:
        text.append(w1)
    if w2 is not None:
        text.append(w2)
    
    if not text:
        w1, w2 = None, None
    elif len(text) == 1:
        w1, w2 = None, text[0]
    else:
        w1, w2 = text[-2] if len(text) >= 2 else None, text[-1]
    
    sentence_finished = False
    iterations_without_word = 0
    
    while not sentence_finished and len(text) < max_len:
        w3 = sample_next(model, w1, w2)
        if w3 is None:
            iterations_without_word += 1
            if iterations_without_word > 3:
                sentence_finished = True
            w1, w2 = None, None
            continue
        
        iterations_without_word = 0
        text.append(w3)
        
        w1, w2 = w2, w3

        if w3 in ['.', '!', '?']:
            sentence_finished = True
    
    return format_sentence(text, capitalize_first=True, add_final_punct=True)

In [21]:
def generate_paragraph(model: dict, n_sentences: int = 3, seeds: Tuple[str, str] = (None, None)) -> str:
    sentences = []
    sentences.append(generate_sentence(model, seeds=seeds))
    for _ in range(n_sentences - 1):
        sentences.append(generate_sentence(model, seeds=(None, None)))
    return " ".join(sentences)

In [22]:
print(generate_paragraph(model_global, n_sentences=3, seeds=("Ayer", "sucedio")))

Ayer sucedio cuando el congreso ni en comercial de barcelona reacciones tras la cual posteriormente retiro durante tres dias es una region del congresista pasion davila planteo en esa ciudad. Israel mato a la sede del ministerio publico january aprovecha la nueva experiencia recibe por correo y por whatsapp nuestro periodico digital enriquecido peru epaper ahora disponible en yape buscanos. El importante partido en mil hectareas de las zapatillas y no en funcion a informacion que permita manejar los vehiculos de seguridad que incluyo audiciones y callbacks el cortometraje an.


In [23]:
print("DEPORTES - Seeds: ('El', 'equipo')")
print(generate_paragraph(models_por_categoria["Deportes"], n_sentences=4, seeds=("El", "equipo")))

DEPORTES - Seeds: ('El', 'equipo')
El equipo esta vez nos atendio gabriel de anos tambien recordo al kaiser no habria problemas al momento de mi para que el centrodelantero obviamente reclamo airado porque sabia que. Los churres se quedan con unidades el otro clasificado es el propio futbolista oscar ibanez en su pintoresca y compleja totalidad se encuentra en plazo de ambos equipos buscaron el. La etapa nacional de paraguay boca juniors de la organizacion del club sino que tambien formo parte del programa digital mano a pablo cepellini el albiceleste que se subiran al. Alianza lima march y ya entro al balompie albiceleste finalizo contamos en que ellos toman la disciplina de surf a su equipo cienciano en el futbol local como en los.


In [24]:
print("POLÍTICA - Seeds: ('El', 'presidente')")
print(generate_paragraph(models_por_categoria["Política"], n_sentences=3, seeds=("El", "presidente")))

POLÍTICA - Seeds: ('El', 'presidente')
El presidente de la margen izquierda del rio amazonas frente a un cargo a su hermano cesar vizcarra recibieron las dosis de vacuna contra la exalcaldesa de lima rechazo la. La presidenta dina boluarte pero su trayectoria revela un claro viraje que no dara un paso clave en el asunto sobre la legalidad por lo que pasa frente a mercado. El consejo de ministros pcm su nombramiento se ajusto al plazo razonable se recuerda alcanza ya una linea de reformas dentro de la direccion de personal tecnico en el area.


In [25]:
print("MODELO GLOBAL - Sin seeds (inicio natural)")
print(generate_paragraph(model_global, n_sentences=3, seeds=(None, None)))

MODELO GLOBAL - Sin seeds (inicio natural)
Al menos fallecidos heridos y de apec es una industria superficial te voy a decretar tres dias para llegar al de los organizadores del evento jeres a nivel de atencion. Tras el empate y derrotas en los proximos cuatro anos dos meses despues y al mismo tiempo que pidio su descargo tomara el gobierno democratico pago prision y se registra. La actriz conocida por su parte el presidente decidira lo que realmente sirva al publico podran destinar un millon al excomico andres hurtado chibolin en la resolucion que disponia de.
